# describe_duration.ipynb

This notebook provides descriptive statistics (median and IQR) of the duration of sick notes associated with long COVID.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns

from functools import reduce
from glob import glob

pd.options.mode.chained_assignment = None

## Import and Clean Data

In [2]:
# Read in and append input files
li = []

for file in glob('../output/cohorts/input*_with_duration.csv'):
    df_temp = pd.read_csv(file, low_memory=False)
    # Creates date variable based on file name
    df_temp['cohort'] = file[24:-18]
    li.append(df_temp)
    
df_input = pd.concat(li, axis=0, ignore_index=False).reset_index(drop=True)

In [3]:
# Columns to subset
subset_cols = ['cohort','age_group','sex',
               'ethnicity','imd','region',
               'first_sick_note_duration']
df_clean = df_input[subset_cols]

_____

## Median & IQR

In [4]:
def compute_med_iqr(path, demo=''):
    if demo == '':
        df_med = df_clean.groupby(
            ['cohort'])[['first_sick_note_duration']].apply(np.nanmedian).reset_index()
        df_pct25 = df_clean.groupby(
            ['cohort'])[['first_sick_note_duration']].apply(lambda x: np.nanpercentile(x,25)).reset_index()
        df_pct75 = df_clean.groupby(
            ['cohort'])[['first_sick_note_duration']].apply(lambda x: np.nanpercentile(x,75)).reset_index()
        dfs = [df_med, df_pct25, df_pct75]
        df_out = reduce(
            lambda left,right: pd.merge(left,right,on='cohort'), dfs
        ).rename(columns={'0_x':'median','0_y':'pct25',0:'pct75'})
        df_out['iqr'] = df_out['pct75']-df_out['pct25']
    else:
        df_med = df_clean.groupby(
            ['cohort',demo])[['first_sick_note_duration']].apply(np.nanmedian).reset_index()
        df_pct25 = df_clean.groupby(
            ['cohort',demo])[['first_sick_note_duration']].apply(lambda x: np.nanpercentile(x,25)).reset_index()
        df_pct75 = df_clean.groupby(
            ['cohort',demo])[['first_sick_note_duration']].apply(lambda x: np.nanpercentile(x,75)).reset_index()
        dfs = [df_med, df_pct25, df_pct75]
        df_out = reduce(
            lambda left,right: pd.merge(left,right,on=['cohort',demo]), dfs
        ).rename(columns={'0_x':'median','0_y':'pct25',0:'pct75'})
        df_out['iqr'] = df_out['pct75']-df_out['pct25']
    Path("../output/tabfig/").mkdir(parents=True, exist_ok=True)
    df_out.to_csv('../output/tabfig/' + path + '.csv', index=False)
    return df_out

In [5]:
# Overall
compute_med_iqr('med_iqr_overall')

,cohort,median,pct25,pct75,iqr
0,covid_2020,271.0,223.0,320.0,97.0
1,general_2019,269.0,222.0,318.0,96.0
2,general_2020,271.0,224.0,322.0,98.0
3,pneumonia_2019,269.0,221.0,318.0,97.0


In [6]:
# Age group
compute_med_iqr('med_iqr_age_group', 'age_group')

,cohort,age_group,median,pct25,pct75,iqr
0,covid_2020,0-17,272.0,222.00,317.75,95.75
1,covid_2020,18-24,270.0,219.25,325.00,105.75
2,covid_2020,25-34,270.0,227.00,319.00,92.00
3,covid_2020,35-44,274.0,229.00,323.00,94.00
4,covid_2020,45-54,268.0,223.50,317.00,93.50
5,covid_2020,55-69,273.0,223.00,324.00,101.00
6,covid_2020,70-79,265.5,221.00,310.00,89.00
7,covid_2020,80+,266.0,220.00,319.00,99.00
8,general_2019,0-17,272.5,222.00,321.00,99.00
9,general_2019,18-24,266.0,224.75,310.25,85.50


In [7]:
# Sex
compute_med_iqr('med_iqr_sex', 'sex')

,cohort,sex,median,pct25,pct75,iqr
0,covid_2020,F,272.0,224.00,319.0,95.00
1,covid_2020,M,269.0,222.25,321.0,98.75
2,general_2019,F,269.0,222.00,317.0,95.00
3,general_2019,M,268.0,222.00,318.0,96.00
4,general_2020,F,270.0,224.00,320.0,96.00
5,general_2020,M,273.0,224.00,323.0,99.00
6,pneumonia_2019,F,270.0,222.00,318.0,96.00
7,pneumonia_2019,M,269.0,220.00,317.0,97.00


In [8]:
# Ethnicity
compute_med_iqr('med_iqr_ethnicity', 'ethnicity')

,cohort,ethnicity,median,pct25,pct75,iqr
0,covid_2020,1.0,271.0,223.00,319.0,96.00
1,covid_2020,3.0,272.0,222.00,321.0,99.00
2,covid_2020,5.0,271.0,225.50,326.0,100.50
3,general_2019,1.0,267.0,221.00,316.0,95.00
4,general_2019,3.0,269.0,223.00,317.0,94.00
5,general_2019,5.0,269.0,220.00,317.0,97.00
6,general_2020,1.0,271.0,225.00,322.0,97.00
7,general_2020,3.0,280.0,228.00,328.0,100.00
8,general_2020,5.0,267.5,218.25,318.0,99.75
9,pneumonia_2019,1.0,269.0,221.00,318.0,97.00


In [9]:
# IMD
compute_med_iqr('med_iqr_imd', 'imd')

,cohort,imd,median,pct25,pct75,iqr
0,covid_2020,0,273.0,227.00,312.75,85.75
1,covid_2020,1,268.0,221.25,317.75,96.50
2,covid_2020,2,269.0,221.00,318.00,97.00
3,covid_2020,3,271.0,227.00,321.00,94.00
4,covid_2020,4,273.0,226.00,325.00,99.00
5,covid_2020,5,270.0,222.00,318.00,96.00
6,general_2019,0,269.0,225.00,315.25,90.25
7,general_2019,1,268.0,220.00,318.00,98.00
8,general_2019,2,267.0,219.50,317.00,97.50
9,general_2019,3,270.0,224.00,314.00,90.00


In [10]:
# Region
compute_med_iqr('med_iqr_region', 'region')

,cohort,region,median,pct25,pct75,iqr
0,covid_2020,East,266.0,220.00,310.50,90.50
1,covid_2020,East Midlands,271.0,226.00,316.00,90.00
2,covid_2020,London,273.0,225.00,319.00,94.00
3,covid_2020,North East,269.0,221.50,320.00,98.50
4,covid_2020,North West,270.0,220.50,320.00,99.50
5,covid_2020,South East,266.0,216.00,316.00,100.00
6,covid_2020,South West,271.0,226.00,321.00,95.00
7,covid_2020,West Midlands,270.0,225.00,324.25,99.25
8,covid_2020,Yorkshire and The Humber,273.0,227.00,323.50,96.50
9,general_2019,East,271.0,220.75,321.00,100.25
